In [12]:
from Policy import Policy
import numpy as np

from Generate_Dataset import Generate_Dataset
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn import MSELoss
import torch
import matplotlib.pyplot as plt
%matplotlib inline


ModuleNotFoundError: No module named 'AI_Models'

In [6]:
DATASET_SIZE = 10000
INPUT_LOWER_LIMIT = 1
INPUT_UPPER_LIMIT = 100
OUTPUT_LOWER_LIMIT = 1
OUTPUT_UPPER_LIMIT = 10000

NUM_EPOCHS = 1000

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
pretrainedCritic = torch.load('../LLM_Critic/pretrainedCritic.pt',map_location=device)
pretrainedCritic.to(device)
randomMarketMovement = np.random.randint(1, 101, size=(DATASET_SIZE, 1)).to_device(device)

X = torch.argmax(pretrainedCritic(randomMarketMovement))
model = Policy(1,1000,1,1000)

/tmp/ipykernel_25111/4094982139.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrainedCritic = torch.load('../LLM_Critic/pretrainedCritic.pt',map_location=device)


AttributeError: 'collections.OrderedDict' object has no attribute 'to'